<a href="https://colab.research.google.com/github/maayan13avraham/colab-assignments/blob/main/assignment_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 0: Set up


**In this assignment we will focus on setting up and get a deeper understanting on the environments we will work with** in practical assignments and in the final class project.


### Goals:

1.   Get familiar with Google Colab and Google Cloud Platform (GCP).
2.   Execute basic code in these environments.
3.   Run tests.

This assignment is **not graded**, its sole purpose is to help you get set up and comfortable in the environments we will use in subsequent assignments and in class project.







### Tasks:


1.   Go through [Colab](https://colab.research.google.com/notebooks/intro.ipynb?utm_source=scs-index) and [GCP](https://www.ee.columbia.edu/~cylin/course/bigdata/HW0_tutorial_1.pdf) tutorials.
2.   Run the following notebook to read a wikipedia file and split it to articles.
3. Write the code to extract the right number of articles.


### Data

In all the coding assigments in the course you will need to use Wikipedia Data. Full data can be found [here](https://dumps.wikimedia.org/enwiki/20210801/), but this data is not preprocessed.
To save you (and us) time, in most assignment, and in the project as well, you will use preprocessed files that can be found in a Google Storage Bucket in the link "gs://wikidata_preprocessed/". When you will need to use data from the google storage bucket, we will provide the code to read the data.  

In [ ]:
import numpy as np
import bz2
from functools import partial
from collections import Counter
import pickle
from itertools import islice
from xml.etree import ElementTree
import codecs
import csv
import time
import os
import re
import gdown
from pathlib import Path

## Read Wikipedia Data


In [ ]:
## Download one wikipedia file
path_url = 'https://drive.google.com/file/d/1c2ggRHG0WqLmJpIE-HfQgJi73kww3Zyc/view?usp=sharing'
wiki_file = 'enwiki-20210801-pages-articles-multistream15.xml-p17324603p17460152.bz2'
id = '1c2ggRHG0WqLmJpIE-HfQgJi73kww3Zyc'
gdown.download(id=id, output=wiki_file, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1c2ggRHG0WqLmJpIE-HfQgJi73kww3Zyc
From (redirected): https://drive.google.com/uc?id=1c2ggRHG0WqLmJpIE-HfQgJi73kww3Zyc&confirm=t&uuid=2e5165c2-0bcb-4f8f-9fee-2356383ec866
To: /content/enwiki-20210801-pages-articles-multistream15.xml-p17324603p17460152.bz2
100%|██████████| 30.4M/30.4M [00:00<00:00, 58.5MB/s]


'enwiki-20210801-pages-articles-multistream15.xml-p17324603p17460152.bz2'

In [ ]:
# Make sure you downloaded the file
!ls

enwiki-20210801-pages-articles-multistream15.xml-p17324603p17460152.bz2
sample_data


In [ ]:
# Print the first 59 lines of the (uncompressed) file
!bzcat $wiki_file | head -n 59

<mediawiki xmlns="http://www.mediawiki.org/xml/export-0.10/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.mediawiki.org/xml/export-0.10/ http://www.mediawiki.org/xml/export-0.10.xsd" version="0.10" xml:lang="en">
  <siteinfo>
    <sitename>Wikipedia</sitename>
    <dbname>enwiki</dbname>
    <base>https://en.wikipedia.org/wiki/Main_Page</base>
    <generator>MediaWiki 1.37.0-wmf.16</generator>
    <case>first-letter</case>
    <namespaces>
      <namespace key="-2" case="first-letter">Media</namespace>
      <namespace key="-1" case="first-letter">Special</namespace>
      <namespace key="0" case="first-letter" />
      <namespace key="1" case="first-letter">Talk</namespace>
      <namespace key="2" case="first-letter">User</namespace>
      <namespace key="3" case="first-letter">User talk</namespace>
      <namespace key="4" case="first-letter">Wikipedia</namespace>
      <namespace key="5" case="first-letter">Wikipedia talk</namespace>
      <n

In [ ]:
def page_iter(wiki_file):
  """ Reads a wiki dump file and create a generator that yields pages.
  Parameters:
  -----------
  wiki_file: str
    A path to wiki dump file.
  Returns:
  --------
  tuple
    containing three elements: article id, title, and body.
  """
  # open compressed bz2 dump file
  with bz2.open(wiki_file, 'rt', encoding='utf-8', errors='ignore') as f_in:
    # Create iterator for xml that yields output when tag closes
    elems = (elem for _, elem in ElementTree.iterparse(f_in, events=("end",)))
    # Consume the first element and extract the xml namespace from it.
    # Although the raw xml has the  short tag names without namespace, i.e. it
    # has <page> tags and not <http://wwww.mediawiki.org/xml/export...:page>
    # tags, the parser reads it *with* the namespace. Therefore, it needs the
    # namespace when looking for child elements in the find function as below.
    elem = next(elems)
    print(elem)
    m = re.match(r"^{(http://www\.mediawiki\.org/xml/export-.*?)}", elem.tag)
    if m is None:
        raise ValueError("Malformed MediaWiki dump")
    ns = {"ns": m.group(1)}
    page_tag = ElementTree.QName(ns['ns'], 'page').text
    # iterate over elements
    for elem in elems:
      if elem.tag == page_tag:
        # Filter out redirect and non-article pages
        if elem.find('./ns:redirect', ns) is not None or \
           elem.find('./ns:ns', ns).text != '0':
          elem.clear()
          continue
        # Extract the article wiki id
        wiki_id = elem.find('./ns:id', ns).text
        # Extract the article title into a variables called title
        # None for now, you will be asked to extract the title in assignment 1.
        title = None
        # extract body
        body = elem.find('./ns:revision/ns:text', ns).text

        yield wiki_id, title, body
        elem.clear()

**YOUR TASK** (Non Graded): Complete the following function that returns the number of pages in the dump file we read. Code parts for implementation are marked with a comment `# YOUR CODE HERE`. The last cell is a test to check your implementation.

In [ ]:
def count_pages():
  """ Returns a count of the number of pages yielded by `page_iter`. """
  count = 0
  pageiter = page_iter(wiki_file)
  for _ in pageiter:
    count +=1

  return count




In [ ]:
assert 10974 == count_pages()

<Element '{http://www.mediawiki.org/xml/export-0.10/}sitename' at 0x7c9f775fd3f0>
